In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
## DEFINE VARIABLES
# CATALOG_URI = "http://nessie-service.nessie-dev.svc.cluster.local:19120/api/v1"
CATALOG_URI = "http://nessie-service.nessie-dev.svc.cluster.local:6788/api/v1"
# WAREHOUSE = "s3://iceberg/"
WAREHOUSE = "s3a://iceberg/"
# STORAGE_URI = "http://minio.minio-dev.svc.cluster.local:9000"
# STORAGE_URI = "http://minio-service.minio-dev.svc.cluster.local:6544"
STORAGE_URI = "http://10.106.91.195:6544"

In [3]:
## CONFIGURE SPARK SESSION
conf = (
    pyspark.SparkConf()
        .setAppName('Iceberg Ingestion')
        .set('spark.jars.packages', 
             'org.postgresql:postgresql:42.7.3,'
             'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,'
             'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.77.1,'
             'software.amazon.awssdk:bundle:2.24.8,'
             'software.amazon.awssdk:url-connection-client:2.24.8')
        .set('spark.sql.extensions', 
             'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,'
             'org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', CATALOG_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.s3.endpoint', STORAGE_URI)
        .set('spark.sql.catalog.nessie.warehouse', WAREHOUSE)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
)

In [4]:
## START SPARK SESSION
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.postgresql#postgresql added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-360ad86e-99ee-489d-95cb-3d1c9ffb70e6;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.0 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.77.1 in central
	found software.amazon.awssdk#bundle;2.24.8 in central
	found software.amazon.awssdk#url-connection-client;2.24.8 in central
	found software.amazon.awssdk#utils;2.24

Spark Running


In [5]:
# Define the JDBC connection properties
jdbc_url = "jdbc:postgresql://postgres.psql-dev.svc.cluster.local:5432/postgres"
properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

In [6]:
sales_df = spark.read.jdbc(url=jdbc_url, table="fashion_sales", properties=properties)

In [7]:
sales_df.show()

+---+---------------+---------+------------+----------+--------------+------------------+-------------+
| id|   product_name| category|sales_amount|sales_date|store_location|customer_age_group|campaign_name|
+---+---------------+---------+------------+----------+--------------+------------------+-------------+
|  1| Slim Fit Jeans|    Denim|       89.99|2024-03-01|      New York|             18-24|Spring Launch|
|  2| Leather Jacket|Outerwear|      249.99|2024-03-01|   Los Angeles|             25-34|Spring Launch|
|  3|Graphic T-Shirt|     Tops|       39.99|2024-03-02|       Chicago|             18-24|March Madness|
|  4|   Summer Dress|  Dresses|      129.99|2024-03-03|      New York|             35-44|March Madness|
|  5|Casual Sneakers| Footwear|       99.99|2024-03-03|   Los Angeles|             25-34|Spring Launch|
+---+---------------+---------+------------+----------+--------------+------------------+-------------+



In [8]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.sales;")

DataFrame[]

In [9]:
sales_df.writeTo("nessie.sales.fashion_sales").createOrReplace()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In [8]:
spark.read.table("nessie.sales.fashion_sales").show()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


+---+---------------+---------+------------+----------+--------------+------------------+-------------+
| id|   product_name| category|sales_amount|sales_date|store_location|customer_age_group|campaign_name|
+---+---------------+---------+------------+----------+--------------+------------------+-------------+
|  1| Slim Fit Jeans|    Denim|       89.99|2024-03-01|      New York|             18-24|Spring Launch|
|  2| Leather Jacket|Outerwear|      249.99|2024-03-01|   Los Angeles|             25-34|Spring Launch|
|  3|Graphic T-Shirt|     Tops|       39.99|2024-03-02|       Chicago|             18-24|March Madness|
|  4|   Summer Dress|  Dresses|      129.99|2024-03-03|      New York|             35-44|March Madness|
|  5|Casual Sneakers| Footwear|       99.99|2024-03-03|   Los Angeles|             25-34|Spring Launch|
+---+---------------+---------+------------+----------+--------------+------------------+-------------+



In [10]:
spark.stop()